# Introduction:

We will be working with a dataset composed of images of hands making a number in different positions, our goal is to be able via an image of a hand indicationg a certain number to recognize that number with a good level of accuracy. 

This will help make machines more sign language friendly. You can find the dataset used for this tutorial in the following link:

https://www.kaggle.com/koryakinp/fingers

The dataset contains 21600 128x128 images of hands making a number sign. 
The train data contains 18000 images and the test data contains 3600 images.

But before getting into business let's do a little warm-up with Tensorflow and explore its main functionalities.

![image info](./1.png) $ y = 1$  ![image info](./0.png) $y = 0$  ![image info](./5.png) $y = 5$ 